# 2. Базовые методы обработка текста

1) Машинное представление текста на естественном языке;  
2) Предобработка текста: токенизация и сегментация;  
3) Нормализация слов: стеммеры, лемматизаторы;  
4) One Hot Encoding и обратный индекс Тf-idf.

### 1) Машинное представление текста на естественном языке 

Представление на уровне символов.  

Если создать строковую переменную, то в памяти компьютера она будет представлена как набор ячеек памяти, содержащих  единицы и нули.

In [1]:
example = 'abcd1234!@#$'
for s in example:
    index = ord(s)
    byte = bin(s.encode()[0])
    print(s, index, byte, sep='\t')

a	97	0b1100001
b	98	0b1100010
c	99	0b1100011
d	100	0b1100100
1	49	0b110001
2	50	0b110010
3	51	0b110011
4	52	0b110100
!	33	0b100001
@	64	0b1000000
#	35	0b100011
$	36	0b100100


Чтобы связать символ с его двоичным представлением, используются кодировки. Про ASCII мы уже упоминали в первой главе. Для кодировки русского текста на ОС Windows часто используют кодировку windows 1251. На ос семейства Linux - utf8. Для кодировки латинских символов требуется один байт на символ, для кириллицы - два байта на символ. 

Если при работе с текстом Вам встретятся крякозябры, то знайте - проблема в неправильной кодировке символов. Компьютер при попытке преобразовать единицы и нули в символ использовал не ту таблицу, поэтому получается набор нечитаемых символов. Кодировку можно сменить в продвинутых текстовых редакторах или средствами python (функции encode и decode).

In [2]:
print('РџСЂРёРІРµС‚'.encode('cp1251').decode('utf-8'))

Привет


Представление на уровне слов.  

Пусть имеется некоторый достаточно большой корпус текста. Мы можем выписать все уникальные слова из этого корпуса, получив подмножество языка, состоящее из нескольких тысяч слов. Некоторые из этих слов не найти в толковом словаре, но в нашем корпусе они встречаются, и что самое важное, они несут в себе определенный смысл. Воспользуемся заранее подготовленным словарем. 

In [3]:
# Read file and strip \n symbols
with open('texts/vocab.txt') as f:
    vocab = f.readlines()
vocab = [s.strip() for s in vocab]

# First 10 words
print(vocab[:10], '...', vocab[-10:], f' Всего в словаре {len(vocab)} слов.')

['a', 'a1', 'a2', 'aa', 'aaa', 'aachen', 'aarhus', 'aaron', 'ab', 'aba'] ... ['zones', 'zoning', 'zoo', 'zoological', 'zoology', 'zoom', 'zu', 'zulu', 'zur', 'zurich']  Всего в словаре 21767 слов.


Простейший метод кодирования слова в языке - это его индекс в словаре.

In [4]:
vocab.index('word')

21516

### 2) Предобработка текста: токенизация и сегментация;

Предложение тоже можно представить как список индексов, для этого его предварительно нужно сегментировать.  

**Сегментация текста** (text segmentation) - это процесс разделения текста на значимые единицы, такие как слова, фразы и предложения.  
**Токенизация** (tokenize) - частный случай сегментации, в котором разделение основано на четком критерии (обычно по определенному символу).  

Например, разделение текста на предложения можно осуществить, используя точку в качестве разделителя. 

In [5]:
text = 'In the town where I was born. Lived a man who sailed to sea'
tokens = text.split('.')
print(tokens)

['In the town where I was born', ' Lived a man who sailed to sea']


Однако такой алгоритм сломается, если в тексте присутствуют сокращения. На помощь приходят специальные библиотеки, которые реализуют сложные правила и обрабатывают исключения. Мы будем использовать библиотеку nltk

Разделение предложения на слова тоже не тривиальная задача, т.к. не во всех естественных языках присутствуют маркеры границ слов, как пробелы в русском и английском. Например, сегментация предложения "Синхронизация разработки и строительства в Пудуне, Шанхай" на китайском языке выглядит вот так:  

'上海浦东开发与建设同步' → ['上海', '浦东', '开发', ‘与', ’建设', '同步']  

К счастью, в этом курсе мы не будем иметь дело с китайским языком.  
Для английского языка простейший способ сегментации предложения на слова - это токенизация по пробелу.

In [6]:
sentense = 'In the town where I was born'.lower()
tokens = sentense.split()
print(tokens)

['in', 'the', 'town', 'where', 'i', 'was', 'born']


In [7]:
sent_indexes = [vocab.index(t) for t in tokens]
print(sent_indexes)

[9677, 19574, 19938, 21294, 9493, 21119, 2308]


Что делать со словами, которых нет в словаре?  
Простейший способ решения этой проблемы - использование в словаре специального слова "< UNK >" (unknown, неизвестный). Тогда если в предложении встретится слово, которого нет в словаре, ему будет присвоен индекс слова < UNK >, например 0.  


### 3) Нормализация слов: стеммеры, лемматизаторы;

В естественных языках одно и то же слово используется в разных падежах и числах. Возникают вопросы: для слова "машина" и "машины" в единственном и множественном числе использовать одну позицию в словаре или разные? Насколько большим будет словарь, если для каждой словоформы использовать отдельный код?

Для решения этой проблемы используется **нормализация** - замена одного слова на другое (нормальное), которое имеет представление в словаре. В общем случае под нормализацией подразумевается две техники: лемматизация и стеммизация. 

**Стемминг** - это простой метод нормализации, чаще всего реализуемый в виде ряда правил, которые постепенно применяются к слову для получения нормализованной формы. Стем - это грубо говоря корень, основа слова. 
Эти правила варьируются от языка к языку и отражают морфологическую структуру используемого языка. Например, для английского языка для преобразование слова в единственную форму возможным правилом может быть удаление буквы “s” в конце.  
Стемминг в основном используется для индексации документов в поисковой системе, поэтому результатом стемминга могут быть недопустимые слова, например *engine -> engin*. Такое допускается, если слово в нормальной форме не отображается пользователю и обработка идет только внутри системы, например для поиска документов.

In [8]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [9]:
stemmer.stem('engines')

'engin'

**Лемматизация** — процесс приведения словоформы к лемме — её нормальной (словарной) форме. Грубо говоря, это более сложная версия стемминга. Лемматизация сводит каждое слово к его надлежащей базовой форме, то есть к слову, которое мы можем найти в словаре.  
Лемматизация работает точнее, но медленнее, чем стемминг. Продвинутые алгоритмы лемматизации используют контестную информацию для определения части речи слова и правило его приведения к нормальной форме.  

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
lemmatizer.lemmatize('engines', pos='n')

'engine'

В русском языке нормальными формами считаются следующие морфологические формы: для существительных — именительный падеж, единственное число; для прилагательных — именительный падеж, единственное число, мужской род; для глаголов, причастий, деепричастий — глагол в инфинитиве (неопределённой форме) несовершенного вида. В отличии от английского, для которого уже придумано множество хороших лемматизаторов, улучшение алгоритмов для русского и других, менее распространенных языков, является активной областью ислледований. 

### 4) One Hot Encoding и обратный индекс Тf-idf.

Использование индекса словаря для представления слов простая, но не самая лучшай техника. По своей природе числа  используются для описания количества чего-либо, а в случае со словами мы имеем дело с разными категориями.  
Довольно примитивный, но более подходящий способ представления слов - использование вектора с размером равным длине словаря. Все элементы этого вектора равны нулю, за исключением позиции, которой соответствует индекс этого слова. (Рисунок OHE). 

<img src='imgs/ohe.jpg'>

Представление на уровне предложений  

Как правило, на практике мы имеем дело с предложениями разной длины, что затрудняет их обработку. Мы можем использовать технику One Hot Encoding для кодирования нескольких слов одного предложения или документа. На выходе будет вектор, длина которого равна количеству слов в словаре, а на позициях слов будет количество их вхождений. Т.е. вектор для предложения вычисляется как сумма OHE векторов каждого слова.  

In [12]:
from sklearn import preprocessing
encoder = preprocessing.OneHotEncoder(categories=[vocab], handle_unknown='ignore')

Т.к. результат содержит очень много нулей и всего одну единицу, то для хранения автоматически используется специальный тип данных - разряженный массив (sparse array).

In [13]:
sparse_ohe = encoder.fit_transform([['радост'],['гор']])
print(sparse_ohe)

In [14]:
sparse_ohe.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

Возникает проблема с наиболее часто встречающимися словами (например, с артиклем the), которые не несут смысловой нагрузки и зашумляют OHE вектор. Для ее решения часто используют две техники:
- выбрасывают стоп-слова
- TF-IDF

Стоп слова - это слова, которые часто встречаются в текстах и используются для связки основных слов. В пакете nltk уже записаны самые распространенные, в т.ч. для русского языка. Нужно только их загрузить.

In [15]:
nltk.download("stopwords")
from nltk.corpus import stopwords
ru_stopwords = stopwords.words("russian")
print(ru_stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
sent = 'мой дядя самых честных правил когда не в шутку занемог'
filt_sent = [t for t in sent.split() if t not in ru_stopwords]
print(filt_sent)

['дядя', 'самых', 'честных', 'правил', 'шутку', 'занемог']


TF (term frequency — частота слова) — отношение числа вхождений некоторого слова к общему числу слов документа. Таким образом, оценивается важность слова $t_{i}$ в пределах отдельного документа. 

$$\mathrm {tf} (t,d)={\frac {n_{t}}{\sum _{k}n_{k}}}$$

где $n_t$ есть число вхождений слова $t$ в документ, а в знаменателе — общее число слов в данном документе.  

IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции. Учёт IDF уменьшает вес широкоупотребительных слов. Для каждого уникального слова в пределах конкретной коллекции документов существует только одно значение IDF. 

$$\mathrm {idf} (t,D)=\log {\frac {|D|}{|\{\,d_{i}\in D\mid t\in d_{i}\,\}|}}$$
где

$|D|$ — число документов в коллекции;  
$ |\{\,d_{i}\in D\mid t\in d_{i}\,\}|$ — число документов из коллекции $D$, в которых встречается $t$ (когда $n_{t}\neq 0$).  

Таким образом, мера TF-IDF является произведением двух сомножителей:

$$ \operatorname {tf-idf}(t,d,D)=\operatorname {tf}(t,d)\times \operatorname {idf}(t,D)$$

Большой вес в TF-IDF получат слова с высокой частотой в пределах конкретного документа и с низкой частотой употреблений в других документах.  
Например, если документ содержит 100 слов, и слово «заяц» встречается в нём 3 раза, то частота слова (TF) для слова «заяц» в документе будет 0,03 (3/100).  
Вычислим IDF как десятичный логарифм отношения количества всех документов к количеству документов, содержащих слово «заяц». Таким образом, если «заяц» содержится в 1000 документах из 10 000 000 документов, то IDF будет равной: log(10 000 000/1000) = 4. Для расчета окончательного значения веса слова необходимо TF умножить на IDF. В данном примере, TF-IDF вес для слова «заяц» в выбранном документе будет равен: 0,03 × 4 = 0,12.  


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


In [18]:
vectorizer.get_feature_names()

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

In [19]:
import numpy as np
np.set_printoptions(precision=2)
print(X.todense())

[[0.   0.47 0.58 0.38 0.   0.   0.38 0.   0.38]
 [0.   0.69 0.   0.28 0.   0.54 0.28 0.   0.28]
 [0.51 0.   0.   0.27 0.51 0.   0.27 0.51 0.27]
 [0.   0.47 0.58 0.38 0.   0.   0.38 0.   0.38]]


### Практическая часть

Используя два набора положительных и отрицательных слов, создадим конвейр для определения тональности сообщения и интегрируем его с чат ботом.

In [20]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("russian") 

In [21]:
with open('texts/positive.txt') as f:
    pos_words = f.readlines()
pos_words = [w.strip() for w in pos_words]
pos_words = [stemmer.stem(w) for w in pos_words] 

with open('texts/negative.txt') as f:
    neg_words = f.readlines()
neg_words = [w.strip() for w in neg_words]
neg_words = [stemmer.stem(w) for w in neg_words] 

vocab = pos_words + neg_words

len(vocab)

249

In [22]:
from nltk.tokenize import word_tokenize

msg = 'привет мой хороший чат бот'
tokens = word_tokenize(msg, language='russian')
filtered_tokens = [t for t in tokens if t not in ru_stopwords]
stem_tokens = [[stemmer.stem(t)] for t in filtered_tokens]

In [23]:
stem_tokens

[['привет'], ['хорош'], ['чат'], ['бот']]

In [24]:
from sklearn.preprocessing import OneHotEncoder
encoder = preprocessing.OneHotEncoder(categories=[vocab], handle_unknown='ignore')

sparse_vec = encoder.fit_transform(stem_tokens)
sent_vec = sparse_vec.sum(axis=0)
sent_vec

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

Определим вектора для положительных и отрицательных слов.

In [25]:
positive_vec = np.zeros((1, len(vocab)))
positive_vec[:, :len(pos_words)] = 1
negative_vec = np.zeros((1, len(vocab)))
negative_vec[:, len(pos_words):] = 1

positive_vec, negative_vec

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

[Скалярное произведение](https://ru.wikipedia.org/wiki/скалярное_произведение) вычисляется как сумма поэлементных произведений двух векторов и в библиотеке numpy имеет свое специальное обозначение - @

In [26]:
sent_vec @ positive_vec.T

matrix([[1.]])

In [27]:
sent_vec @ negative_vec.T

matrix([[0.]])

In [28]:
def get_answer(msg):
    tokens = word_tokenize(msg, language='russian')
    filtered_tokens = [t for t in tokens if t not in ru_stopwords]
    stem_tokens = [[stemmer.stem(t)] for t in filtered_tokens]
    sparse_vec = encoder.fit_transform(stem_tokens)
    msg_vec = sparse_vec.sum(axis=0)
    pos_score = msg_vec @ positive_vec.T
    neg_score = msg_vec @ negative_vec.T
    print(pos_score, neg_score)
    if pos_score > neg_score:
        return 'Это хорошо'
    elif neg_score > pos_score:
        return 'Это плохо'
    else:
        return 'Это нормально'

Протестируем нашу функцию

In [29]:
get_answer('голод и горечь')

[[0.]] [[2.]]


'Это плохо'

In [30]:
get_answer('весна и веселье')

[[2.]] [[0.]]


'Это хорошо'

In [31]:
import telebot

TOKEN = '5441620452:AAGQQtoC8ohgdsvsOUO7ubSD6y86oRQ-hL0'

bot = telebot.TeleBot(TOKEN) 

@bot.message_handler(content_types=['text'])
def send_echo(message):
    answer = get_answer(message.text)
    
    bot.send_message(message.chat.id, answer)

bot.polling(none_stop=True)

[[0.]] [[1.]]


Ссылки: 
- https://nlpub.ru
- https://habr.com/ru/company/Voximplant/blog/446738/
- Hobson Lane etc, NLP in action, Глава 2. 
- https://wordsonline.ru/samples/